<a href="https://colab.research.google.com/github/harshaharod21/Conversational-Voice-ChatBot/blob/main/voice_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install openai==0.10.2
!pip install pydub
!pip install audio_recorder_streamlit


In [ ]:
# Set your OpenAI API key here
OPENAI_API_KEY = 'sk-proj-AFWgm83prP5u4zbLizqoT3BlbkFJdSdncWx9ch6oIsYtDQWt'


In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = OPENAI_API_KEY

In [ ]:
import streamlit as st
import os
from utils import get_answer, text_to_speech, speech_to_text
from audio_recorder_streamlit import audio_recorder


In [ ]:
# Initialize session state for managing chat messages
def initialize_session_state():
    if "messages" not in st.session_state:
        st.session_state.messages = [{"role": "assistant", "content": "Hi! How may I assist you today?"}]

initialize_session_state()

st.title("OpenAI Conversational Chatbot 🤖")

# Handling Voice Inputs
footer_container = st.container()
with footer_container:
    audio_bytes = audio_recorder()
if audio_bytes:
    with st.spinner("Transcribing..."):
        webm_file_path = "temp_audio.mp3"
        with open(webm_file_path, "wb") as f:
            f.write(audio_bytes)

        transcript = speech_to_text(webm_file_path)
        if transcript:
            st.session_state.messages.append({"role": "user", "content": transcript})
            with st.chat_message("user"):
                st.write(transcript)
            os.remove(webm_file_path)

# Chatbot Response Processing
if st.session_state.messages[-1]["role"] != "assistant":
    with st.chat_message("assistant"):
        with st.spinner("Thinking🤔..."):
            final_response = get_answer(st.session_state.messages)
        with st.spinner("Generating audio response..."):
            audio_file = text_to_speech(final_response)
            st.audio(audio_file, format='audio/mp3')
        st.write(final_response)
        st.session_state.messages.append({"role": "assistant", "content": final_response})
        os.remove(audio_file)